In [1]:
# Project
import os
print(os.getcwd())
import pandas as pd
import numpy as np
import seaborn as sns
import csv
import matplotlib.pyplot as plt
import random as random
import statsmodels.api as sm
from linearmodels import PanelOLS
from linearmodels import RandomEffects
import statsmodels.formula.api as smf
# Imports functions from seperate notebook
%run Functions.ipynb

C:\Users\Matt\Downloads\CMSE project


Here I load in the data and spit out some summary statistics.

In [2]:
# Reading in Data
# Keys to extract formatted State and County names
key = pd.read_excel('fips_codes_website.xls')   # Counties and major cities
key00 = pd.read_csv('FIPS_countycodes.txt',delimiter=',')  # Counties only
key00['GU Name'] = key00['GU Name'].str[:-7]  # Slice 'county' from names
# Housing Data
zillow2019 = pd.read_excel("County_Zhvi_Summary_AllHomes2019.xls")
zillow2019['RegionName'] = zillow2019['RegionName'].str[:-7]  # Slice 'county' from names
# Crime
crime2010 = pd.read_csv('33523-0001-crimedata2010.txt',delimiter='\t')
crime2012 = pd.read_csv('35019-0001-crimedata2012.txt',delimiter='\t')
crime2014 = pd.read_csv('36399-0001-crimedata2014.txt',delimiter='\t')
crime2016 = pd.read_csv('37059-0001-crimedata2016.txt',delimiter='\t')
# Enumployment & Wages
#test_wages_18 = pd.read_excel('allhlcn183.xlsx',sheet_name='US_St_Cn_MSA')
wage_unemp2018 = pd.read_csv('allhlcn183wage_unemp2018.txt',delimiter='\t',dtype={'St':str})
wage_unemp2016 = pd.read_csv('allhlcn163wage_unemp2016.txt',delimiter='\t',dtype={'St':str})
wage_unemp2014 = pd.read_csv('allhlcn143wage_unemp2014.txt',delimiter='\t',dtype={'St':str})
wage_unemp2012 = pd.read_csv('allhlcn123wage_unemp2012.txt',delimiter='\t',dtype={'St':str})
wage_unemp2010 = pd.read_csv('allhlcn103wage_unemp2010.txt',delimiter='\t',dtype={'St':str})

print("2014 crime:",crime2014.shape,"// key:",key.shape)
print("2019 Housing:",zillow2019.shape)
print('2018 Wages:',wage_unemp2018.shape)
x = ((crime2014.shape[0])*(crime2014.shape[1]))*4 + ((wage_unemp2018.shape[0])*(wage_unemp2018.shape[1]))*5 + (key00.shape[0])*key00.shape[1]
print(x,'unique datapoints in all datasets')

2014 crime: (3177, 56) // key: (41787, 7)
2019 Housing: (1921, 17)
2018 Wages: (62724, 21)
7313843 unique datapoints in all datasets


In [3]:
# Data Cleaning #
# Yearly cleaned databases
crime10 = crime_cleaner(crime2010)
crime12 = crime_cleaner(crime2012)
crime14 = crime_cleaner(crime2014)
crime16 = crime_cleaner(crime2016)
cols = list(crime14.columns.values) # Grab columns
# Cleaned Wage Data
economic10 = wage_cleaner(wage_unemp2010) # Returns 3 items; county, state, national, and urban level data sets
economic12 = wage_cleaner(wage_unemp2012)
economic14 = wage_cleaner(wage_unemp2014)
economic16 = wage_cleaner(wage_unemp2016)
economic18 = wage_cleaner(wage_unemp2018)
economic18['Industry'] = economic18['Industry'].str[1:] # Remove first space in 2018 industry codes

# Combined crime and economic dataframes and rearrange columns
db2010 = crime10.merge(economic10,left_on=['State Abbreviation','GU Name'],right_on=['State Abbreviation','GU Name'])
cols = db2010.columns.tolist()  # Code to rearrange columns
cols = ['Year']+cols[:-14]+cols[-10:]
db2010 = db2010[cols]
db2012 = crime12.merge(economic12,left_on=['State Abbreviation','GU Name'],right_on=['State Abbreviation','GU Name'])
db2012 = db2012[cols]
db2014 = crime14.merge(economic14,left_on=['State Abbreviation','GU Name'],right_on=['State Abbreviation','GU Name'])
db2014 = db2014[cols]
db2016 = crime16.merge(economic16,left_on=['State Abbreviation','GU Name'],right_on=['State Abbreviation','GU Name'])
db2016 = db2016[cols]

C:\Users\Matt\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Matt\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Matt\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [5]:
# Selects 10 Random counties and stores them as a dictionary by count // Key = MI, County
samp_db = {}
for n in range(200):
    r = random.randint(0,len(crime12))
    x = crime12['State Abbreviation'].iloc[r] # Pull State Abreviation
    y = crime12['GU Name'].iloc[r]  # Pull analogous county name
    z = x+', '+y # Format name for dictionary key
    agg = search(y,x) # Return criminal and economic data
    samp_db[z] = agg  # Add data to empty dictionary
keys = list(samp_db.keys())
print(len(samp_db))
print(samp_db.keys())

194
dict_keys(['NE, Valley', 'CO, Boulder', 'MN, Steele', 'CO, Park', 'TX, Collin', 'MN, Beltrami', 'KY, Estill', 'TN, Blount', 'KS, Greenwood', 'VA, Buena Vis', 'OK, Le Flore', 'LA, Rapides', 'VA, Prince Edward', 'WV, Nicholas', 'PA, Butler', 'TN, Union', 'SC, Aiken', 'ND, McKenzie', 'GA, Newton', 'MI, Presque Isle', 'NE, Webster', 'CA, Mariposa', 'LA, Avoyelles', 'KY, Ohio', 'AL, Houston', 'CT, Middlesex', 'LA, Caldwell', 'KS, Wichita', 'OK, Blaine', 'IL, Ogle', 'PA, Allegheny', 'IL, Scott', 'NY, Essex', 'OH, Scioto', 'TX, Floyd', 'KS, Haskell', 'MT, Sweet Grass', 'ND, Mountrail', 'MT, Yellowstone', 'NE, Fillmore', 'GA, Habersham', 'NE, Furnas', 'VA, Sussex', 'GA, Troup', 'NC, Pamlico', 'MI, Jackson', 'WI, Price', 'PA, Venango', 'FL, Leon', 'TX, Brewster', 'PA, Chester', 'KS, Leavenworth', 'ND, Cavalier', 'NJ, Warren', 'CO, San Juan', 'IN, Marshall', 'TX, Hemphill', 'IN, Lake', 'PA, Washington', 'TX, Matagorda', 'OR, Polk', 'OR, Multnomah', 'WV, Pendleton', 'NY, St. Lawrence', 'NE, L

In [6]:
industry = most_establishments(samp_db[keys[1]][3])  # Returns 4 largest industries in 2016 ,index = 3
industry = list(industry['Industry']) # List of largest industries for reference purposes
#print(samp_db[keys[1]][3]['Industry'])
industry = ['Goods-producing','Service-providing','Trade, transportation, and utilities','Professional and business services']

**REGRESSIONS**

Here I figured out how to [sort of] do multiple regression for a single county. It runs, however the numbers are junk, as indicated by the near perfect R-squared.

In [7]:
samp_dic = {}
stat = 'Establishment Count','Average Weekly Wage','September Employment'
# Gather //  Reformat economic data
for s in stat:
    for n in industry:
        samp_dic[n+' '+s] = industrydata_cleaner(samp_db,s,n)
# Gather // Reformat criminal data
crimes = samp_db[keys[1]][0].columns[:48] # Reset crimes to grab all crime data
for i in crimes:
    samp_dic[i] = sample_crime_cleaner(samp_db,i)

In [8]:
finaldb = pd.DataFrame(data=samp_dic)
# Reorganize columns
cols = finaldb.columns.tolist()
cols = cols[12:15]+cols[:12]+cols[15:]
finaldb = finaldb[cols]
# Convert economic data from arrays to ints
for i in cols[3:15]:
    finaldb[i] = finaldb[i].astype(float)
print('\nFinal Database for all 10 counties over 4 years in my sample.')
finaldb

ValueError: setting an array element with a sequence.

In [9]:
finaldb

,Year,State Abbreviation,GU Name,Goods-producing Establishment Count,Service-providing Establishment Count,"Trade, transportation, and utilities Establishment Count",Professional and business services Establishment Count,Goods-producing Average Weekly Wage,Service-providing Average Weekly Wage,"Trade, transportation, and utilities Average Weekly Wage",...,OFAGFAM,DUI,LIQUOR,DRUNK,DISORDR,VAGRANT,ALLOTHR,SUSPICN,CURFEW,RUNAWAY
0,2010,NE,Valley,[46],[131],[48],[16],[484],[429],[462],...,0,0,1,0,0,0,0,0,0,0
1,2012,NE,Valley,[48],[143],[44],[16],[500],[458],[499],...,0,4,2,0,0,0,0,0,0,0
2,2014,NE,Valley,[61],[150],[45],[16],[573],[513],[518],...,0,4,11,0,3,0,16,0,0,0
3,2016,NE,Valley,[69],[150],[46],[17],[660],[565],[597],...,0,3,7,0,1,0,47,0,0,0
4,2010,CO,Boulder,[1524],[11365],[2160],[4069],[1233],[1011],[760],...,176,1903,1366,1546,429,3,6467,0,77,0
5,2012,CO,Boulder,[1441],[11748],[2175],[4297],[1245],[1055],[797],...,141,1767,1764,1180,272,0,7251,0,77,0
6,2014,CO,Boulder,[1478],[12188],[2155],[4477],[1261],[1112],[845],...,146,1515,1150,0,265,1,6443,0,89,7
7,2016,CO,Boulder,[1534],[13138],[2236],[4913],[1415],[1191],[945],...,183,1301,1122,0,194,1,7767,0,47,1
8,2010,MN,Steele,[194],[734],[265],[92],[960],[596],[558],...,7,168,162,0,82,0,121,0,0,3
9,2012,MN,Steele,[203],[717],[253],[89],[899],[553],[556],...,6,159,116,0,61,0,106,0,0,0


In [ ]:
x = finaldb[list(finaldb.columns.values[3:15])]  # Use only the first 3 columns to regress
y = finaldb['DRGPOSS']  # Set y variable

model = sm.OLS(y, x).fit()
predictions = model.predict(x) # make the predictions by the model

print(model.summary())

In [ ]:
d = {'col1': [1,'',6], 'col2': [3,4,7], 'col3': [3,4,7]}
df = pd.DataFrame(data=d)
df

In [ ]:
df = df.replace('',np.nan)
df = df.dropna(axis=0,how='any')
df